In [60]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [63]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [66]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 491 ms, sys: 1.43 ms, total: 492 ms
Wall time: 491 ms


In [67]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 302 ms, sys: 2.46 ms, total: 304 ms
Wall time: 303 ms


In [75]:
%%time 
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 138 ms, sys: 8.32 ms, total: 146 ms
Wall time: 143 ms


0       -38.114105
1       -38.125487
2       -38.084228
3       -38.065981
4       -38.045047
           ...    
27926   -38.106420
27927   -38.113307
27928   -38.121112
27929   -38.108484
27930   -38.151707
Length: 27931, dtype: float64

In [77]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 1.21 ms, sys: 346 μs, total: 1.55 ms
Wall time: 1.62 ms


0       -38.114105
1       -38.125487
2       -38.084228
3       -38.065981
4       -38.045047
           ...    
27926   -38.106420
27927   -38.113307
27928   -38.121112
27929   -38.108484
27930   -38.151707
Length: 27931, dtype: float64

# Practice

In [80]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
941,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
14013,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [108]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 279     EVENING  OTHERS  MOTOR VEHICLE THEFT   
         1308   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         4675        DAY  OTHERS         THEFT F/AUTO   
         11621   EVENING  OTHERS          THEFT/OTHER   
         15616       DAY  OTHERS          THEFT/OTHER   

                                                     BLOCK    XBLOCK  \
20165648 279            1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         1308             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         4675          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         11621  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15616   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 279    137478.0  ...       10603.0    Precinct 144  38.905159   
         1308   135959.0  ...        8200.0     Precinct 89  38.891475   
         4675   137456.0  ...       10602.0     Precinct 83  38.904961   
         11621  136899.0  ...        5900.0    Precinct 143  38.899942   
         15616  135255.0  ...        6500.0     Precinct 89  38.885133   

                LONGITUDE           BID              START_DATE  \
20165648 279   -77.005891          NOMA  2020/11/20 20:15:26+00   
         1308  -76.999516           NaN  2020/11/20 03:02:27+00   
         4675  -76.997314           NaN  2020/11/19 22:30:39+00   
         11621 -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         15616 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 279    2020/11/20 21:46:24+00  596093043             NaN   
         1308                      NaN  596105470             NaN   
         4675   2020/11/20 03:00:43+00  596362501             NaN   
         11621  2020/11/20 22:08:28+00  596387673             NaN   
         15616                     NaN  596402829             NaN   

                               report_dt  
20165648 279   2020-11-20 22:17:27+00:00  
         1308  2020-11-20 04:27:36+00:00  
         4675  2020-11-20 15:06:04+00:00  
         11621 2020-11-20 22:07:10+00:00  
         15616 2020-11-20 12:46:32+00:00  

[5 rows x 26 columns]

In [98]:
#shift by mean
df.X - df.X.mean(
)

0       -0.026244
1        0.001648
2        0.057401
3       -0.020910
4        0.061127
           ...   
27926    0.032075
27927   -0.024906
27928   -0.013876
27929    0.005722
27930   -0.046272
Name: X, Length: 27931, dtype: float64

In [100]:
#shift down by day
r.report_dt - timedelta(days=1)

Timestamp('2020-08-28 05:00:25+0000', tz='UTC')

In [94]:
same_wards

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
10,-77.025989,38.904912,20178124,2020/12/16 18:58:23+00,DAY,OTHERS,THEFT F/AUTO,1100 - 1199 BLOCK OF 10TH STREET NW,397746.0,137450.0,...,4902.0,Precinct 129,38.904904,-77.025986,NaN,2020/12/16 00:30:34+00,2020/12/16 13:30:43+00,596090470,NaN,2020-12-16 18:58:23+00:00
14,-77.020912,38.893040,20113277,2020/08/08 21:33:15+00,EVENING,OTHERS,THEFT/OTHER,600 - 699 BLOCK OF PENNSYLVANIA AVENUE NW,398186.0,136132.0,...,5801.0,Precinct 143,38.893032,-77.020910,DOWNTOWN,2020/08/08 20:54:47+00,2020/08/08 20:55:24+00,596090498,NaN,2020-08-08 21:33:15+00:00
19,-77.031121,38.909135,20184032,2020/12/29 22:54:16+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF RHODE ISLAND AVENUE NW,397301.0,137919.0,...,5003.0,Precinct 17,38.909128,-77.031119,NaN,2020/12/29 21:20:50+00,2020/12/29 21:30:31+00,596090538,NaN,2020-12-29 22:54:16+00:00
20,-77.038520,38.903134,20087049,2020/06/12 19:38:25+00,EVENING,OTHERS,THEFT/OTHER,1000 - 1099 BLOCK OF 17TH STREET NW,396659.0,137253.0,...,10700.0,Precinct 17,38.903126,-77.038517,GOLDEN TRIANGLE,2020/06/12 18:05:51+00,2020/06/12 18:06:40+00,596090539,NaN,2020-06-12 19:38:25+00:00
24,-77.027810,38.904209,20171059,2020/12/02 00:03:51+00,EVENING,GUN,ROBBERY,1106 - 1199 BLOCK OF MASSACHUSETTS AVENUE NW,397588.0,137372.0,...,5004.0,Precinct 17,38.904201,-77.027808,NaN,2020/12/01 19:00:35+00,2020/12/01 19:15:30+00,596090574,NaN,2020-12-02 00:03:51+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27911,-77.050775,38.903742,20055056,2020/04/05 05:28:13+00,MIDNIGHT,OTHERS,THEFT F/AUTO,2300 - 2399 BLOCK OF L STREET NW,395596.0,137321.0,...,5501.0,Precinct 4,38.903734,-77.050773,NaN,2020/04/05 05:08:06+00,2020/04/05 05:32:33+00,596642640,NaN,2020-04-05 05:28:13+00:00
27921,-77.036548,38.903134,20400767,2020/06/12 21:16:16+00,EVENING,OTHERS,THEFT/OTHER,1000 - 1099 BLOCK OF 16TH STREET NW,396830.0,137253.0,...,10700.0,Precinct 17,38.903126,-77.036546,NaN,2020/06/12 17:30:00+00,2020/06/12 18:00:00+00,596642724,NaN,2020-06-12 21:16:16+00:00
27925,-77.043456,38.906060,20075289,2020/05/21 16:52:54+00,DAY,OTHERS,THEFT F/AUTO,1200 - 1219 BLOCK OF 19TH STREET NW,396231.0,137578.0,...,10700.0,Precinct 17,38.906052,-77.043453,GOLDEN TRIANGLE,2020/05/20 23:30:09+00,2020/05/21 00:00:20+00,596642762,NaN,2020-05-21 16:52:54+00:00
27928,-77.020926,38.899814,20157380,2020/11/03 20:43:43+00,EVENING,OTHERS,THEFT/OTHER,600 - 699 BLOCK OF H STREET NW,398185.0,136884.0,...,5801.0,Precinct 129,38.899806,-77.020924,DOWNTOWN,2020/11/03 19:20:06+00,2020/11/03 20:43:53+00,596642770,NaN,2020-11-03 20:43:43+00:00


# 1.5 Iterrow Approach

In [89]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 734   -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         9297  -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
         14014 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         14015 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         18683 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 734    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         9297   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         14014      DAY  OTHERS         THEFT F/AUTO   
         14015      DAY  OTHERS         THEFT F/AUTO   
         18683      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 734         1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         9297        1100 - 1199 BLOCK OF F STREET NW  397609.0  136611.0   
         14014       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         14015  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         18683       700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 734    ...       10700.0     Precinct 17  38.905648 -77.039822   
         9297   ...        5802.0    Precinct 129  38.897346 -77.027563   
         14014  ...        5303.0     Precinct 15  38.909638 -77.040089   
         14015  ...        4100.0     Precinct 13  38.913346 -77.050526   
         18683  ...        5801.0    Precinct 129  38.899121 -77.021926   

                            BID              START_DATE  \
20123250 734    GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         9297          DOWNTOWN  2020/08/29 23:08:57+00   
         14014              NaN  2020/08/28 22:00:23+00   
         14015              NaN  2020/08/26 22:00:29+00   
         18683         DOWNTOWN  2020/08/29 16:05:40+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 734    2020/08/29 19:00:05+00  596095073             NaN   
         9297                      NaN  596376108             NaN   
         14014  2020/08/29 08:00:27+00  596397251             NaN   
         14015  2020/08/27 12:00:51+00  596397252             NaN   
         18683  2020/08/29 16:08:33+00  596447637             NaN   

                               report_dt  
20123250 734   2020-08-29 22:04:46+00:00  
         9297  2020-08-30 00:05:52+00:00  
         14014 2020-08-29 16:05:18+00:00  
         14015 2020-08-29 16:45:57+00:00  
         18683 2020-08-29 17:15:19+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [155]:
## create empty container to store results 
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    #Subset C_Other on the ward of C_Target
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    #Create a time cutoff of time of crime + 1200 minutes
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    #Create a time filter
    time_filter = (same_wards.report_dt >= r.report_dt) & (same_wards.report_dt <= CUTOFF)
    #Filter the subset by time as well (between time of crime and cutoff) Store data in dictionary
    same_wards_sametime = same_wards[time_filter].copy()
    store_matches_2[str(r.CCN)] = same_wards_sametime

### 2.2 apply it to one of the focal crimes

In [153]:
#iloc[0] --> first row
r = C_Tar.iloc[0]
find_related_crimes(r)
store_matches_2

{'20165648':                X          Y       CCN              REPORT_DAT     SHIFT  \
 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   EVENING   
 1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00  MIDNIGHT   
 4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00       DAY   
 11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   EVENING   
 15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00       DAY   
 15617 -76.997316  38.904969  20165803  2020/11/20 14:45:06+00       DAY   
 16271 -76.994365  38.900203  20165859  2020/11/20 15:37:59+00       DAY   
 25170 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00       DAY   
 
        METHOD              OFFENSE  \
 279    OTHERS  MOTOR VEHICLE THEFT   
 1308   OTHERS  MOTOR VEHICLE THEFT   
 4675   OTHERS         THEFT F/AUTO   
 11621  OTHERS          THEFT/OTHER   
 15616  OTHERS          THEFT/OTHER   
 15617  OTHERS         THEFT F/AUTO   
 16271  OTHERS          THEFT

### 2.3 Use apply to cover all the other focal crimes

In [163]:
#Apply find_focal_crimes on every row (axis = 1)
C_Tar.apply(find_related_crimes, axis = 1)
# returns None because the function doesnt' return anything

941      None
14013    None
dtype: object

In [165]:
all_matches_2 = pd.concat(store_matches_2) #smash two dataframes into one dataframe
all_matches_2

X          Y       CCN              REPORT_DAT  \
20165648 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         15617 -76.997316  38.904969  20165803  2020/11/20 14:45:06+00   
         16271 -76.994365  38.900203  20165859  2020/11/20 15:37:59+00   
         25170 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   
20123250 734   -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         9297  -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
         14014 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         14015 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         18683 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         18803 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 279     EVENING  OTHERS  MOTOR VEHICLE THEFT   
         1308   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         4675        DAY  OTHERS         THEFT F/AUTO   
         11621   EVENING  OTHERS          THEFT/OTHER   
         15616       DAY  OTHERS          THEFT/OTHER   
         15617       DAY  OTHERS         THEFT F/AUTO   
         16271       DAY  OTHERS          THEFT/OTHER   
         25170       DAY  OTHERS         THEFT F/AUTO   
20123250 734     EVENING  OTHERS  MOTOR VEHICLE THEFT   
         9297    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         14014       DAY  OTHERS         THEFT F/AUTO   
         14015       DAY  OTHERS         THEFT F/AUTO   
         18683       DAY  OTHERS          THEFT/OTHER   
         18803       DAY  OTHERS          THEFT/OTHER   

                                                     BLOCK    XBLOCK  \
20165648 279            1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         1308             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         4675          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         11621  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15616   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         15617         600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         16271              800 - 899 BLOCK OF H STREET NE  400489.0   
         25170              300 - 399 BLOCK OF G STREET NE  399886.0   
20123250 734              1700 - 1779 BLOCK OF M STREET NW  396546.0   
         9297             1100 - 1199 BLOCK OF F STREET NW  397609.0   
         14014            1700 - 1799 BLOCK OF P STREET NW  396523.0   
         14015       2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0   
         18683            700 - 799 BLOCK OF 7TH STREET NW  398098.0   
         18803         1724 - 1799 BLOCK OF 17TH STREET NW  396662.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 279    137478.0  ...       10603.0    Precinct 144  38.905159   
         1308   135959.0  ...        8200.0     Precinct 89  38.891475   
         4675   137456.0  ...       10602.0     Precinct 83  38.904961   
         11621  136899.0  ...        5900.0    Precinct 143  38.899942   
         15616  135255.0  ...        6500.0     Precinct 89  38.885133   
         15617  137456.0  ...       10602.0     Precinct 83  38.904961   
         16271  136927.0  ...        8402.0     Precinct 82  38.900195   
         25170  136784.0  ...        8301.0     Precinct 83  38.898907   
20123250 734    137533.0  ...       10700.0     Precinct 17  38.905648   
         9297   136611.0  ...        5802.0    Precinct 129  38.897346   
         14014  137976.0  ...        5303.0     Precinct 15  38.909638   
         14015  138388.0  ...        4100.0     Precinct 13  38.913346   
         18683

### On the dataframes

Pseudocode:  
Dictionary to store data  
find_focal_crimes( df1_search_in, df2_search_for, timedelta)  
New_df is subset of df1 that is the ward of df2  
Cutoff is the search datetime plus the timedelta Filter new_df by time  
Store data in dictionary  
Apply find_focal_crimes on the two dataframes  

### Wrapping

In [168]:
C_Tar

,CCN,WARD,OFFENSE,report_dt
941,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
14013,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


In [174]:
C_Oth.head(n=2)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
0,-77.033293,38.919188,20118678,2020/08/20 00:24:28+00,EVENING,OTHERS,THEFT/OTHER,1400 - 1499 BLOCK OF W STREET NW,397113.00,139035.00,...,4300.0,Precinct 22,38.919180,-77.033291,NaN,2020/08/08 00:24:44+00,2020/08/20 00:24:46+00,596090405,NaN,2020-08-20 00:24:28+00:00
1,-77.005401,38.879914,20023507,2020/02/08 05:59:18+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,800 - 899 BLOCK OF NEW JERSEY AVE SE,399531.54,134674.78,...,7203.0,Precinct 131,38.879907,-77.005399,CAPITOL RIVERFRONT,2020/02/08 01:43:11+00,2020/02/08 01:53:11+00,596090413,NaN,2020-02-08 05:59:18+00:00


In [184]:
store_matches_3 = {}
def find_cases_by_df(C_Tar, C_Oth, delta):
    for i,r in C_Tar.iterrows():
     same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
     CUTOFF = r.report_dt +  timedelta(minutes=delta)
     time_filter = (same_wards.report_dt >= r.report_dt) & (same_wards.report_dt <= CUTOFF)
     same_wards_sametime = same_wards[time_filter].copy()
    
     store_matches_3[str(r.CCN)] = same_wards_sametime